In [3]:
import threading
import time
import natural.date

import ipywidgets as widgets


class gui:
    def __init__(self, replay_callback, save_callback, load_ttn_callback, load_gpx_callback):
        self.w_live = widgets.Checkbox(value=True, description='Live update', disabled=False, indent=False)
        self.w_last_seen = widgets.Label('Last seen: Never')
        
        self.w_msg_slider = widgets.IntSlider(value=0, min=0, max=0, step=1, description='Replay:', disabled=False, continuous_update=True, orientation='horizontal', readout=True, readout_format='d', layout=widgets.Layout(width='100%'))
        self.w_msg_slider.observe(self.slide_event)
        self.w_save_button = widgets.Button(description='Save TTN file')
        self.w_save_button.on_click(self.save_event)
        self.w_load_ttn_file = widgets.FileUpload(accept='.pickle', multiple=False, description='Load TTN file')
        self.w_load_ttn_file.observe(self.upload_ttn_event)
        self.w_load_gpx_file = widgets.FileUpload(accept='.gpx', multiple=False, description='Load GPX file')
        self.w_load_gpx_file.observe(self.upload_gpx_event)
        
        self.w_first_hbox = widgets.HBox([self.w_last_seen, self.w_live])
        self.w_second_hbox = widgets.HBox([self.w_load_ttn_file, self.w_load_gpx_file, self.w_save_button])
        self.last_seen = None
        
        display(widgets.VBox([self.w_first_hbox, self.w_msg_slider, self.w_second_hbox]))
        
        self.update_active = True
        self.replay_callback = replay_callback
        self.save_callback = save_callback
        self.load_ttn_callback = load_ttn_callback
        self.load_gpx_callback = load_gpx_callback
        threading.Thread(target=self.last_seen_thread).start()
        
    def __del__(self):
            self.update_active = False
        
    def last_seen_thread(self):
        while self.update_active:
            if self.last_seen != None:
                self.w_last_seen.value = 'Last seen: ' + natural.date.duration(self.last_seen) + ' (' + time.ctime(self.last_seen) + ')'
            time.sleep(1)
            
    def update(self, num_msg, curr_msg_idx, timestamp):
        self.last_seen = timestamp
        self.w_msg_slider.max = num_msg-1
        self.w_msg_slider.value = curr_msg_idx
        
    def slide_event(self, change):
        if change['name'] == 'value':
            self.replay_callback(self.w_msg_slider.value)
            
    def upload_ttn_event(self, change):
        if change['name'] == 'value':
            self.load_ttn_callback(list(self.w_load_ttn_file.value.values())[0]['content'])
    
    def upload_gpx_event(self, change):
        if change['name'] == 'value':
            self.load_gpx_callback(list(self.w_load_gpx_file.value.values())[0]['content'].decode("utf-8"))

    def save_event(self, button):
        self.save_callback()